**Задание**

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

Будем пытаться генерировать новые токены. Для обучения возьмем Властелина колец. 

In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=d68832222ae20fd198501ddb79d71e77ff797cfba6cc4601b5f4be94904c9375
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [16]:
import tensorflow as tf

import numpy as np
import os
import time

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from string import punctuation
from stop_words import get_stop_words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
import io
import chardet
import os
import codecs

filename = '/content/lordoftherings.txt'

bytes = min(32, os.path.getsize(filename))
raw = open(filename, 'rb').read(bytes)

if raw.startswith(codecs.BOM_UTF8):
    encoding = 'utf-8-sig'
else:
    result = chardet.detect(raw)
    encoding = result['encoding']

infile = io.open(filename, 'r', encoding=encoding)
text = infile.read()
infile.close()

In [15]:
print(text[:500])

Дж. Р. Р. Толкиен

ВЛАСТЕЛИН КОЛЕЦ


Перевод В. С. Муравьева, А. А. Кистяковского

		Три Кольца – для царственных эльфов в небесных шатрах,
		Семь – для властителей гномов, гранильщиков в каменном лоне,
		Девять – для Девятерых, облеченных в могильный прах,
		Одно наденет Владыка на черном троне,
		В стране по имени Мордор, где распростерся мрак.
		Одно Кольцо покорит их, одно соберет их,
		Одно их притянет и в черную цепь скует их
		В стране по имени Мордор, где распростерся мрак.



ПРОЛОГ






Предобработка текста

In [17]:
sw = get_stop_words("ru") + stopwords.words('russian')
punkt = list(punctuation)
noise = set(sw + punkt)

In [18]:
def make_tokens(text):
    return [token for token in word_tokenize(text) if token not in noise and token.isalpha()]
    
tokens = make_tokens(text)

In [19]:
vocab = sorted(set(tokens))
len(vocab)

49481

In [20]:
vocab[-10:]

['ятаган',
 'ятаганами',
 'ятаганом',
 'ятаганы',
 'ячеистой',
 'ячмень',
 'ящеркой',
 'ящик',
 'ящика',
 'ящиками']

In [21]:
# Задаем отображения из множества уникальных букв в множество индексов
token2idx = {u:i for i, u in enumerate(vocab)} # словарь {индекс: токен}

idx2token = np.array(vocab) # array из символов словаря (можно по индексу извлекать токен)

In [22]:
# Теперь переводим все символы текста в индексы - на выходе array из индексов
text_as_int = np.array([token2idx[c] for c in tokens])

In [24]:
print(text[:300]), print(text_as_int[:300])

Дж. Р. Р. Толкиен

ВЛАСТЕЛИН КОЛЕЦ


Перевод В. С. Муравьева, А. А. Кистяковского

		Три Кольца – для царственных эльфов в небесных шатрах,
		Семь – для властителей гномов, гранильщиков в каменном лоне,
		Девять – для Девятерых, облеченных в могильный прах,
		Одно наденет Владыка на черном троне,
		
[ 1705  6625   557  2592  4520  3484  2695  6719  2760 48024 49203 24693
 48750  5889 10336 13041 13508 19579 21903  1653  1650 26355 22957 34543
  4093 23787   826 48376 45282   552 43661 19057  3408 38733 23252  4093
  2768 32675 27233 42111  4093 35981 48388 48170 41329   552 43661 19057
  3408 38733 23252  4441  3961  7108  5587 32517 28263 47724 22107 48502
 22893 46244 26025 19381  5748 22108 29388 31276 23868  7147 30457 24609
 12833    39  2722  2199  5088 24154   318  6650 11177 36915 40311 24311
  8827 22820  1300 25982 26882 31776  6754 26669 31275 43687   318 11096
 10606 14805  2613 22762   318 47741 45939 39910 21276 13602 42184 34803
 31187  3329 10934 47727 18414 47886 18405

(None, None)

In [25]:
# Максимальная длина предложения (максимальное количество слов в батче) для входных данных (в буквах)
tokens_length = 200

# Количество эпох
examples_per_epoch = len(text)//(tokens_length+1)

# создаем экземпляр датасета из идексов токенов
tokens_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in tokens_dataset.take(5):
    print(idx2token[i.numpy()], len(tokens_dataset))

Дж 205224
Толкиен 205224
ВЛАСТЕЛИН 205224
КОЛЕЦ 205224
Перевод 205224


In [26]:
# Создаем последовательность из батчей токенов с установленной длиной батча
sequences = tokens_dataset.batch(tokens_length+1, drop_remainder = True)

In [27]:
# Разбиваем каждый батч на признаки и целевую переменную (следующую букву)
def split_input_target(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам -> таким образом формируем новый датасет
dataset = sequences.map(split_input_target)

Нейросеть

In [28]:
# размер батча
BATCH_SIZE = 16

# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# перемешивание разделенных данных
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [29]:
# Длина словаря
vocab_size = len(vocab)

# Длина выходного эмбеддинга
embedding_dim = 512

# Количество скрытых состояний в RNN слое 
rnn_units = 1024

In [30]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [31]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape," # (batch_size, sequence_length, vocab_size)")

(16, 200, 49481)  # (batch_size, sequence_length, vocab_size)


In [33]:
# сэмплируем предсказание 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([23447,  2872, 20708, 48516, 43328, 39287,  8670, 15511, 38477,
       30062, 29915, 40042,  6949,   960, 36532,  2656, 19318, 28447,
       11308, 37551,  1417,  7163, 42493, 18300, 37652,  8330,  4869,
       42804, 48624,   851, 42748,  5927, 26138, 29928, 43396,  6803,
       40340, 18151,  7822, 11278, 35133,  8398,  7868,  6699,    27,
       49196, 27679, 12331, 22718,  8212, 31042, 45595, 48401, 33260,
       43333, 37760, 35099, 48804, 15318, 31594, 13835, 12758, 43973,
        5377, 40604,  3687, 24725, 40304, 47129, 47667, 30310,    28,
       34638, 37684,  4104, 35996, 43103,  8204, 21030, 29991, 15191,
        1162, 49092, 45484, 36230, 25462, 30434, 19027, 33362,  5994,
       28648, 11244,  9328,  2582, 41399, 28409, 10353,  9839, 29247,
       25723, 16906, 16467, 29240, 47042, 31311, 32122,  7987, 32415,
       45655, 46891, 27993,  2564, 18933, 26829, 18259, 44610, 10651,
       16769, 12748, 25953,  5932, 34474,  1031, 13305, 43868, 48430,
       44615,  3237,

In [34]:
print("Input: \n", repr(" ".join(idx2token[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2token[sampled_indices ])))

Input: 
 'взгляд невысокий худощавый Он уловил отсвет ясных серых глаз вздрогнул внезапно поняв лице надежды затеняет смерть Серая дорога вела вдоль берега Снеговой бурлящей каменьях селений Ундерхерга Обернана скорбные женские лица выглядывали темных дверей войско провожали путь рога арфы поющие голоса Так начался великий поход восток котором слагали песни многие поколения ристанийцев Из темного Дунхерга тусклое утро Вывел сын Тенгела последнюю рать И достиг Эдораса царственные чертоги Старинные златоверхие застланы мглой Здесь обители предков распростился народом Со своим свободным народом очагом своим Простился высоким троном благословенным кровом Под которым пировал былые светлые дни И поехал конунг пятам гнался ужас Впереди ожидал рок Но присягал верность Он принес нерушимую клятву исполнил Теоден ехал Пять дней ночей Мчались эорлинги вперед восточным путем Через Фольд Фенмарк Фириэнвуд Шесть копьеносцев мчались Санлендинг К могучей твердыне Мундбург горы Миндоллуин К столице Госу

In [35]:
# функция потерь

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# подсчитаем ошибку
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (16, 200, 49481)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.809343


In [36]:
# место для хранения checkpoint
checkpoint_dir = '/content/training_checkpoints'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=9*30, # сохраняем каждую 30-ю итерацию 
    save_weights_only=True, # будем сохранять только веса
    )

In [39]:
# Компиляция модели
model.compile(optimizer='adam', loss=loss)

#обучение модели
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
63/63 [==============================] - 42s 595ms/step - loss: 9.6931
Epoch 2/50
63/63 [==============================] - 38s 604ms/step - loss: 9.6011
Epoch 3/50
63/63 [==============================] - 44s 696ms/step - loss: 9.5861
Epoch 4/50
63/63 [==============================] - 38s 607ms/step - loss: 9.5761
Epoch 5/50
63/63 [==============================] - 38s 599ms/step - loss: 9.5777
Epoch 6/50
63/63 [==============================] - 38s 601ms/step - loss: 9.5717
Epoch 7/50
63/63 [==============================] - 43s 680ms/step - loss: 9.5682
Epoch 8/50
63/63 [==============================] - 39s 609ms/step - loss: 9.5640
Epoch 9/50
63/63 [==============================] - 38s 600ms/step - loss: 9.5650
Epoch 10/50
63/63 [==============================] - 38s 602ms/step - loss: 9.5647
Epoch 11/50
63/63 [==============================] - 43s 681ms/step - loss: 9.5647
Epoch 12/50
63/63 [==============================] - 38s 605ms/step - loss: 9.5609
Epoch 13/50
6

In [40]:
#предсказание модели
example_batch_predictions = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 200, 49481) # (batch_size, sequence_length, vocab_size)


In [41]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [42]:
print("Input: \n", repr("".join(idx2token[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2token[sampled_indices ])))

Input: 
 'взглядневысокийхудощавыйОнуловилотсветясныхсерыхглазвздрогнулвнезапнопонявлиценадеждызатеняетсмертьСераядорогавелавдольберегаСнеговойбурлящейкаменьяхселенийУндерхергаОбернанаскорбныеженскиелицавыглядывалитемныхдверейвойскопровожалипутьрогаарфыпоющиеголосаТакначалсявеликийпоходвостоккоторомслагалипеснимногиепоколенияристанийцевИзтемногоДунхергатусклоеутроВывелсынТенгелапоследнююратьИдостигЭдорасацарственныечертогиСтаринныезлатоверхиезастланымглойЗдесьобителипредковраспростилсянародомСосвоимсвободнымнародомочагомсвоимПростилсявысокимтрономблагословеннымкровомПодкоторымпировалбылыесветлыедниИпоехалконунгпятамгналсяужасВпередиожидалрокНоприсягалверностьОнпринеснерушимуюклятвуисполнилТеоденехалПятьднейночейМчалисьэорлингивпередвосточнымпутемЧерезФольдФенмаркФириэнвудШестькопьеносцевмчалисьСанлендингКмогучейтвердынеМундбурггорыМиндоллуинКстолицеГосударейМоряприплывшихТеперьосажденнойврагамиокруженнойогнемСудьбаторопилатемнотапоглотилаПоглотилаконейконниковстукуходящихкопытЗаглохтиш

Генерация текста

In [43]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'/content/training_checkpoints/ckpt_50'

In [44]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [45]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число токенов для генераци
    num_generate = 10

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [token2idx[s] for s in make_tokens(start_string)]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)
    # print(input_eval)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.5

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):

        #получаем предсказания модели
        predictions = model(input_eval)

        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)

        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        # predictions = predictions / temperature

        # выберем последний токен из отсэмплированных предсказаний[-1], т.к. именно он будет предсказанным следующим токеном в строке
        # индекс 0 - выбираем именно индекс токена (помимо него выводится еще размер [1]и тип [2])
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        # print(predicted_id)

        # Передаем предсказанный символ в качестве следующего ввода в модель, по нему будет предсказывать следующий символ
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2token[predicted_id])

    return (start_string + ' '.join(text_generated))

In [49]:
text_ = generate_text(model, start_string=u"Эта повесть начинается ")
print(text_)

Эта повесть начинается спокойно рассеивали пределами пестрый спокойно Гондор Однако держитесь видения вроде
